In [ ]:
!pip install -q -U kaggle_environments

In [ ]:
import random
from kaggle_environments import make, evaluate


# Добавляем агентов

#1. Выбирает случайное действие: 0 (камень), 1 (бумага) или 2 (ножницы).

In [ ]:
class Random_agent:
   def __call__(self, observation, configuration):
    return random.randint(0, 2)


#2. Показываем всегда камень

In [ ]:
class Rock_agent:
   def __call__(self, observation, configuration):
    return 0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#3. Показываем всегда бумагу

In [ ]:
class Paper_agent:
   def __call__(self, observation, configuration):
    return 1


#4. Показываем всегда ножницы

In [ ]:
class Scissors_agent:
   def __call__(self, observation, configuration):
    return 2

#5. Копирует ход противника
Агент повторяет ход противника, если это не первый ход.
В первом раунде выбирает случайно.

In [ ]:
class Copy_opponent_agent:
   def __call__(self, observation, configuration):
     #Если первый ход - выбираем случайно
    if observation.step == 0:
      return random.randint(0, 2)
    #Выбираем последнее действие противника
    return observation.lastOpponentAction

#6. Играет против последнего хода противника
Агент выбирает действие, побеждающее последний ход противника. В первом раунде выбирает случайно.

In [ ]:
class Counter_opponent_agent:
   def __call__(self, observation, configuration):
     #Если первый ход - выбираем случайно
    if observation.step == 0:
        return random.randint(0, 2)
    return (observation.lastOpponentAction + 1) % 3

# 7. Выбирает наиболее частый ход противника
Агент анализирует прошлые ходы противника и выбирает ход, побеждающий наиболее частый ход противника.

In [ ]:
class Frequency_counter_agent:
    def __init__(self):
      # инициализируем переменную для хранения статистики
        self.lastOpponentActions = []

    def __call__ (self, observation, configuration):
      #Если первый ход - выбираем случайно
      if observation.step == 0:
        return random.randint(0, 2)

      #Запоминаем все предыдущие шаги
      if observation.step > 0:
        self.lastOpponentActions.append(observation.lastOpponentAction)

      counts = [0, 0, 0]

      for action in self.lastOpponentActions:
       #Считаем предыдущие ходы противника
        counts[action] += 1
      #Находим наиболее частый и выбираем ход который его победил бы
      most_frequent = counts.index(max(counts))
      return (most_frequent + 1) % 3


# 8. Случайный со смещением
 Агент с вероятностью 70% выбирает случайный ход и с вероятностью 30%, меняет стратегию на выигрышный ответ против последнего хода.
    

In [ ]:
class Random_shift_agent:
   def __call__(self, observation, configuration):
    if random.random() < 0.7:
        return random.randint(0, 2)
    if observation.step == 0:
        return random.randint(0, 2)
    return (observation.lastOpponentAction + 1) % 3

# 9. Цикл
Агент циклически меняет свои действия: камень -> ножницы -> бумага.

In [ ]:
class Cycle_agent:
   def __call__(self, observation, configuration):
    return observation.step % 3

#10. Зеркало

Агент повторяет ход противника из предыдущего раунда. Если первый раунд — выбирает случайно.

In [ ]:
class Mirror_agent:
   def __call__(self, observation, configuration):
  #Если первый ход - выбираем случайно
    if observation.step == 0:
        return random.randint(0, 2)
        #Повторяем последний ход противника
    return observation.lastOpponentAction



# 11. Анализ тенденций
Агент анализирует последние 5 ходов противника и пытается предугадать следующий ход.

In [ ]:
class Trend_analyzer_agent:
    def __init__(self):
      # инициализируем переменную для хранения статистики
        self.lastOpponentActions = []

    def __call__(self, observation, configuration):
      if observation.step == 0:
        return random.randint(0, 2)
      # Добавляем последний ход противника в историю
      if observation.step > 0:
        self.lastOpponentActions.append(observation.lastOpponentAction)

      last_five = self.lastOpponentActions[-5:]
      #Если было меньше 5 ходов - выбираем случайно
      if len(last_five) < 5:
        return random.randint(0, 2)
      #Пытаемся вычислить тренд
      trend = sum((a - b) for a, b in zip(last_five[1:], last_five[:-1]))
      predicted = (observation.lastOpponentAction + (1 if trend > 0 else -1)) % 3
      return (predicted + 1) % 3

# 12. Агрессивный
Агент чаще всего выбирает бумагу (например, 70% времени), но изредка меняет выбор на камень или ножницы.

In [ ]:
class Aggressive_agent:
  def __call__(self, observation, configuration):
    probabilities = [0.2, 0.7, 0.1]  # Вероятности для камня, бумаги и ножниц
    return random.choices([0, 1, 2], weights=probabilities, k=1)[0]

#13. Реактивный
Пытается предугадать следующий ход противника на основе частоты действий за последние раунды. Агент анализирует частоту действий противника за последние 5 раундов и выбирает ход, который побеждает наиболее частый выбор.

In [ ]:
class Reactive_agent:
    def __init__(self):
      # инициализируем переменную для хранения статистики
        self.lastOpponentActions = []
    def __call__(self, observation, configuration):
      if observation.step == 0:
        return random.randint(0, 2)

      # Добавляем последний ход противника в историю
      if observation.step > 0:
        self.lastOpponentActions.append(observation.lastOpponentAction)

      counts = [0, 0, 0]
      recent_actions = self.lastOpponentActions[-5:]  # Последние 5 действий
      for action in recent_actions:
          counts[action] += 1
      most_frequent = counts.index(max(counts))
      return (most_frequent + 1) % 3

#14. Адаптивный
Анализирует, как противник реагирует на его действия, и адаптируется, чтобы выиграть чаще.
Агент анализирует, как противник реагирует на его действия, и пытается адаптироваться. Использует стратегию на основе прошлых раундов, чтобы предсказать следующий ход.

In [ ]:
class Adaptive_agent:
    def __init__(self):
      # инициализируем переменную для хранения статистики
        self.lastOpponentActions = []
    def __call__(self, observation, configuration):
      if observation.step == 0:
        return random.randint(0, 2)

      # Добавляем последний ход противника в историю
      if observation.step > 0:
        self.lastOpponentActions.append(observation.lastOpponentAction)

    # Сохраняем историю своих действий и действий противника
      my_last_action = self.lastOpponentActions[-1] if observation.step > 1 else random.randint(0, 2)
      opponent_reaction = observation.lastOpponentAction

    # Если противник часто побеждает, меняем свою стратегию
      if opponent_reaction == (my_last_action + 1) % 3:
          return (my_last_action + 2) % 3
      return (opponent_reaction + 1) % 3

# Инциализируем среду и агентов


In [ ]:
# Инициализация среды
env = make("rps", configuration={"episodeSteps": 100}, debug=True)

# Список агентов
agents = {
    "Random": Random_agent(),
    "Rock": Rock_agent(),
    "Paper": Paper_agent(),
    "Scissors": Scissors_agent(),
    "CopyOpponent": Copy_opponent_agent(),
    "CounterOpponent": Counter_opponent_agent(),
    "FrequencyCounter": Frequency_counter_agent(),
    "RandomShift": Random_shift_agent(),
    "Cycle": Cycle_agent(),
    "Mirror": Mirror_agent(),
    "TrendAnalyzer": Trend_analyzer_agent(),
    "Aggressive": Aggressive_agent(),
    "Reactive": Reactive_agent(),
    "Adaptive": Adaptive_agent(),
}

#Функция запуска турнира и подсчета очков

Если агент выиграл даём 2 очка, если ничья то каждому 1 очко, в остальном 0

In [ ]:
# Функция для запуска турнира
def run_tournament(agents):
    scores = {agent: 0 for agent in agents.keys()}
    agent_names = list(agents.keys())

    for i, agent1_name in enumerate(agent_names):
        for agent2_name in agent_names[i+1:]:
            # Запускаем игру между двумя агентами
            result = env.run([agents[agent1_name], agents[agent2_name]])
            rewards = [result[0][0]["reward"], result[-1][1]["reward"]]

                # Обновляем очки
            if rewards[0] > rewards[1]:
              scores[agent1_name] += 2
            elif rewards[0] < rewards[1]:
              scores[agent2_name] += 2
            else:
              scores[agent1_name] += 1
              scores[agent2_name] += 1

    return scores

    # Запуск турнира
tournament_scores = run_tournament(agents)


#Вывод результатов


In [ ]:
scoresTable = sorted(tournament_scores.items(), key=lambda x: -x[1])

# Вывод результатов
print("Результаты турнира:")
for agent, score in scoresTable:
    print(f"{agent}: {score} очков")

print("============================", f"Результативная стратегия: {scoresTable[0][0]}", sep="\n")

Результаты турнира:
Adaptive: 22 очков
CounterOpponent: 19 очков
RandomShift: 17 очков
Reactive: 17 очков
Cycle: 15 очков
Random: 13 очков
FrequencyCounter: 12 очков
Mirror: 11 очков
Scissors: 10 очков
TrendAnalyzer: 10 очков
Aggressive: 10 очков
Paper: 9 очков
CopyOpponent: 9 очков
Rock: 8 очков
Результативная стратегия: Adaptive
